In [ ]:

import os
import pickle

from modules.logging.format_utils import format_measures
from modules.collecting.results_collector import DataFrameCollector
from modules.logging.logger import DefaultLogger
from modules.algorithms.base.OSLPP import Params
from modules.selection.uncertanties import SelectRejectMode
from modules.algorithms.nn.OSLPP_NN_UTILS import train_osda

In [ ]:
with open('experiments/configs/small_datasets.pkl', 'rb') as f:
    config = pickle.load(f)

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [ ]:
from modules.augmentation.transformations import Augmentation

select_reject_mode = SelectRejectMode.CONFIDENCE
logger = DefaultLogger()
for (source, target), (common, tgt_private) in config.items():
    for transform in [Augmentation.TRANSLATE, Augmentation.ROTATE, Augmentation.FLIP, Augmentation.SCALE, Augmentation.AFFINE_DEGREES, Augmentation.BRIGHTNESS, Augmentation.CONTRAST, Augmentation.BLUR]:
        for epochs in [10]:
            for lr in [1e-3]:
                for n_r in [0.15]:
                    for seed in range(1):
                        params = Params(pca_dim=512, proj_dim=128, T=10, n_r=n_r, n_r_ratio=None,
                                        dataset='DomainNet_DCC', source=source, target=target,
                                        num_common=len(common), num_src_priv=0, num_tgt_priv=len(tgt_private))
                        metrics = train_osda(params, lr, epochs, 1, select_reject_mode, seed, common, tgt_private, logger, augmentation_test=True, transform=transform)

In [ ]:
logger.df.to_csv('aug_tests.csv', header=True, index=False)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('aug_tests.csv')

In [ ]:
df['diff_common-private'] = df['similarity_common'] - df['similarity_private']

In [ ]:
import numpy as np

df['name'] = df['source'].apply(lambda x: x[0].upper()) + df['target'].apply(lambda x: x[0].upper())
df.pivot_table(index=['transform'], columns='name', values='diff_common-private',
               aggfunc=[lambda x: ",".join(f"{round(np.mean(x), 2)}".split("."))]).sort_index(key=lambda x: x, ascending=False)

In [ ]:
df['no_aug__diff_conf_common-private'] = df['no_aug_common_conf'] - df['no_aug_private_conf']
df['no_aug__diff_margin_common-private'] = df['no_aug_common_margin'] - df['no_aug_private_margin']
df['no_aug__diff_inv_entropy_common-private'] = df['no_aug_common_inv_entropy'] - df['no_aug_private_inv_entropy']

In [ ]:
import numpy as np

df['name'] = df['source'].apply(lambda x: x[0].upper()) + df['target'].apply(lambda x: x[0].upper())
df.pivot_table(index=['transform'], columns='name', values='no_aug__diff_inv_entropy_common-private',
               aggfunc=[lambda x: ",".join(f"{round(np.mean(x), 2)}".split("."))]).sort_index(key=lambda x: x, ascending=False)